In [2]:
import numpy as np
import ctypes
import time
import os
import sys
import copy
import matplotlib.pyplot as plt
import logging

_import_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if _import_path not in sys.path:
    sys.path.insert(0, _import_path)

# hardware
import hardware.afg3000 as tek
import pyvisa
import hardware.hameg as ha
#import hardware.sr_lockin as lo

from nidaq_device_modTR import NidaqDevice
from configuration_parser import load_config, ConfigIntegrityException
from measurement_worker import MeasurementWorker

In [3]:
log_formatter = logging.Formatter("%(asctime)s [%(levelname)s]  %(message)s")
logger = logging.getLogger('measurement_logger')

handlers_registered = False
if not handlers_registered:
    handlers_registered = True
    
    file_handler = logging.FileHandler(os.path.join(os.getcwd(), 'log.log'))
    file_handler.setFormatter(log_formatter)
    logger.addHandler(file_handler)

    console_handler = logging.StreamHandler()
    console_handler.setFormatter(log_formatter)
    logger.addHandler(console_handler)

logger.setLevel(logging.INFO)
logger.propagate = False

In [14]:
configure_folder_path = os.path.join(os.getcwd(), 'configs')
configure_filenames = os.listdir(configure_folder_path)

for config_filename in configure_filenames:
    config_filepath = os.path.join(configure_folder_path, config_filename)
    try:
        logger.info('Read config file: {}'.format(config_filepath))
        cfg = load_config(config_filepath)
    except ConfigIntegrityException:
        # TODO: add logfile
        exc_info = sys.exc_info()
        raise exc_info[0], exc_info[1], exc_info[2]
    
    try:
        hameg = ha.HMP2030(device="hameg01", voltage_max=20., current_max=0.07)
        tektronix = tek.AFG3252("TCPIP0::129.69.46.235::inst0::INSTR")
        #lock_in = lo.SR830("GPIB0::08::INSTR")

        use_lock_in = cfg['measurement']['lock_in_params']['active']
        nidaq_config = cfg['devices']['nidaq']
        nidaq = NidaqDevice()
        nidaq.load_config(nidaq_config, use_lock_in=use_lock_in)
        mw = MeasurementWorker(nidaq=nidaq, tektronix=tektronix, hameg=hameg, logger=logger)
        mw.set_config(cfg) # load measurement config
        mw.set_base_path(os.path.join('n:', os.sep, 'data', 'dani', 'magnetometer'))

        mw.measure(save=True, config_path=config_filepath)

    except Exception, e:
    
        # TODO: add logfile
        exc_info = sys.exc_info()
        raise exc_info[0], exc_info[1], exc_info[2]
    finally:
        hameg.close()


2016-01-27 11:21:11,397 [INFO]  Read config file: N:\meassrc\magnetometer\new_meas02\configs\config-06.yaml
2016-01-27 11:21:12,092 [INFO]  B1: set freq to 2000.0. dev: tektronix, ch: 2
2016-01-27 11:21:14,223 [INFO]  B1: set freq to 2500.0. dev: tektronix, ch: 2
2016-01-27 11:21:16,819 [INFO]  B1: set freq to 3000.0. dev: tektronix, ch: 2
2016-01-27 11:21:18,947 [INFO]  B1: set freq to 3500.0. dev: tektronix, ch: 2
2016-01-27 11:21:21,078 [INFO]  B1: set freq to 4000.0. dev: tektronix, ch: 2
2016-01-27 11:21:23,289 [INFO]  B1: set freq to 4500.0. dev: tektronix, ch: 2
2016-01-27 11:21:25,420 [INFO]  B1: set freq to 5000.0. dev: tektronix, ch: 2
2016-01-27 11:21:27,542 [INFO]  B1: set freq to 5500.0. dev: tektronix, ch: 2


In [11]:
import copy
a = ['a1', 'a2', 'a3']
b = ['b1', 'b2', 'b3']
c = ['c1', 'c2', 'c3']
x = [a, b, c]

def rec(x, level=0):
    if len(x) == 0:
        return
    xnew = copy.copy(x)
    r = xnew.pop()
    for i in r:
        print str(i) + ' level=' + str(level)
        rec(xnew, level=level+1)
        
x.reverse()
rec(x)

a1 level=0
b1 level=1
c1 level=2
c2 level=2
c3 level=2
b2 level=1
c1 level=2
c2 level=2
c3 level=2
b3 level=1
c1 level=2
c2 level=2
c3 level=2
a2 level=0
b1 level=1
c1 level=2
c2 level=2
c3 level=2
b2 level=1
c1 level=2
c2 level=2
c3 level=2
b3 level=1
c1 level=2
c2 level=2
c3 level=2
a3 level=0
b1 level=1
c1 level=2
c2 level=2
c3 level=2
b2 level=1
c1 level=2
c2 level=2
c3 level=2
b3 level=1
c1 level=2
c2 level=2
c3 level=2
